# Catanomics - Computer Vision and pre-processing
The goal of this notebook is to read in an image of a catan board without number or settlements, and draw the board (ports not included) over the image

In [541]:
# Import necessary libraries
import cv2
import numpy as np

In [542]:
# Show image function for later use
def showImage(img, name=None):
    if not name:
        cv2.imshow("Image display", img)
    else:
        cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# Save image function
def saveImage(filename, img, dir):
    # Get full path
    full_path = f"{dir}/{filename}"
    cv2.imwrite(full_path, img)
    print(f"Image saved to {full_path}")

## 1. Read in the image, detect the border, and find the max/min y point of the permiter

In [543]:
# Read in an image
# image = cv2.imread('../images/v0/catan01q.jpg')
image = cv2.imread('../images/v4/board08.jpg')
# The input imgs are too big, so reduce to 25%
image = cv2.resize(image, (1052,1052))
showImage(image)
og_img = image.copy()

In [544]:
# Convert the image to hsv
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [545]:
# Define color bounds for the perimiter of the board
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

# # This is for beige instead of blue (EXPERIMENT)
# upper_blue = np.array([174, 242, 251])
# lower_blue = np.array([49, 145, 174])

# Create a binary mask where the blue regions are white, and everything else is black
mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
showImage(mask)

In [546]:
# Bitwise AND to keep the blue parts of the image
blue_regions = cv2.bitwise_and(image, image, mask=mask)

In [547]:
# Perform canny edge detection on the blue_regions

# Convert to grayscale
gray_blue_region = cv2.cvtColor(blue_regions, cv2.COLOR_BGR2GRAY)
# Apply Gaussian Blur
blurred_blue_regions = cv2.GaussianBlur(gray_blue_region, (5,5), 0)
# Canny edge detection
edges = cv2.Canny(blurred_blue_regions, 50, 150)
# Show the result
# showImage(edges, "Edges in blue regions")

# 729.24435816, 601.65764162
print(edges)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 2. Perform Hough Transform on the images

In [548]:
# Perform Hough Transform using `HoughLines`
lines = []
thresh = 200
print(len(lines))
while len(lines) < 10:
    lines = cv2.HoughLines(edges, 1, np.pi / 180, thresh)
    print(len(lines))
    print(f"Trying threshold {thresh} | Got {len(lines)} lines.")
    if len(lines) >= 10:
        break
    else:
        thresh -= 5
print(f"Results: {thresh} threshold | {len(lines)} lines")

0
2
Trying threshold 200 | Got 2 lines.
3
Trying threshold 195 | Got 3 lines.
3
Trying threshold 190 | Got 3 lines.
3
Trying threshold 185 | Got 3 lines.
3
Trying threshold 180 | Got 3 lines.
3
Trying threshold 175 | Got 3 lines.
4
Trying threshold 170 | Got 4 lines.
4
Trying threshold 165 | Got 4 lines.
4
Trying threshold 160 | Got 4 lines.
4
Trying threshold 155 | Got 4 lines.
4
Trying threshold 150 | Got 4 lines.
5
Trying threshold 145 | Got 5 lines.
5
Trying threshold 140 | Got 5 lines.
6
Trying threshold 135 | Got 6 lines.
6
Trying threshold 130 | Got 6 lines.
6
Trying threshold 125 | Got 6 lines.
6
Trying threshold 120 | Got 6 lines.
6
Trying threshold 115 | Got 6 lines.
6
Trying threshold 110 | Got 6 lines.
8
Trying threshold 105 | Got 8 lines.
9
Trying threshold 100 | Got 9 lines.
12
Trying threshold 95 | Got 12 lines.
Results: 95 threshold | 12 lines


In [549]:
# Draw lines on the image
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    
showImage(image)
# saveImage("perimiter00.jpg", image, "../images/perimiter/v1")

In [550]:
len(lines)

12

In [551]:
min_rho_difference = 10
overlapping_indexes = set()

while len(lines) - len(overlapping_indexes) > 7:
    for i in range(len(lines)):
        rho, _ = lines[i, 0]
        for j in range(i+1, len(lines)):
            rho2, _ = lines[j, 0]
            if abs(rho - rho2) < min_rho_difference:
                overlapping_indexes.add(j)
    min_rho_difference+=1
    # print(len(overlapping_indexes))
print(f"Overlapping indexes: {overlapping_indexes}")

Overlapping indexes: {6, 7, 8, 9, 10}


In [552]:
perimiter_lines = []
for i in range(len(lines)):
    if i not in overlapping_indexes:
        perimiter_lines.append(lines[i])
        
print(len(perimiter_lines))

7


In [572]:
# This is a list of lists of tuples which are coordinates for each line
line_coords = []
# Calculate start and end points for the lines
for index, line in enumerate(perimiter_lines):
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(og_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    print(f"Coordinates of line {index}: ({x1, y1}) to ({x2, y2})")
    line_coords.append([(x1, y1), (x2, y2)])
line_coords[0]
showImage(og_img)

Coordinates of line 0: ((-1068, 184)) to ((879, 634))
Coordinates of line 1: ((-860, 815)) to ((1102, 433))
Coordinates of line 2: ((-965, 440)) to ((1027, 266))
Coordinates of line 3: ((-1020, 143)) to ((968, 352))
Coordinates of line 4: ((-646, -803)) to ((950, 399))
Coordinates of line 5: ((-636, 869)) to ((1060, -190))
Coordinates of line 6: ((-1000, 329)) to ((999, 330))


In [554]:
import math
# Function to calculate the slope given two points
def slope(x1,y1,x2,y2):
    ###finding slope
    if x2!=x1:
        return((y2-y1)/(x2-x1))
    else:
        return 'NA'

# Function to calculate the y-intercept given two points 
def y_intercept(x1,y1,x2,y2):
    # y = mx+b OR b = y-mx
    b = y1 - int(slope(x1, y1, x2, y2) * x1)
    return b

def calc_intersection(m1,b1,m2,b2):
    # Create the coefficient matrices
    a = np.array([[-m1, 1], [-m2, 1]])
    b = np.array([b1, b2])
    try:
        solution = np.linalg.solve(a, b)
    except:
        solution = (0,0)

    return solution

# Function that draws the lines in the bounds of the image
def drawLine(image,x1,y1,x2,y2):

    m=slope(x1,y1,x2,y2)
    h,w=image.shape[:2]
    if m!='NA':
        ## here we are essentially extending the line to x=0 and x=width
        ## and calculating the y associated with it
        # starting point
        px=0
        py=-(x1-0)*m+y1
        # ending point
        qx=w
        qy=-(x2-w)*m+y2
    else:
    # if slope is zero, draw a line with x=x1 and y=0 and y=height
        px,py=x1,0
        qx,qy=x1,h
    # Draws a green line
    cv2.line(image, (int(px), int(py)), (int(qx), int(qy)), (0, 255, 0), 3)

In [555]:
test_image = og_img.copy()
# List of lists of tuples of slope and y-intercept of linear lines
line_equations = []
print(line_coords)
for line in line_coords:
    try:
        line_equations.append((slope(line[0][0], line[0][1], line[1][0], line[1][1]), y_intercept(line[0][0], line[0][1], line[1][0], line[1][1])))
    except:
        pass
line_equations

[[(-1068, 184), (879, 634)], [(-860, 815), (1102, 433)], [(-965, 440), (1027, 266)], [(-1020, 143), (968, 352)], [(-646, -803), (950, 399)], [(-636, 869), (1060, -190)], [(-1000, 329), (999, 330)]]


[(0.23112480739599384, 430),
 (-0.1946992864424057, 648),
 (-0.08734939759036145, 356),
 (0.10513078470824949, 250),
 (0.7531328320802005, -317),
 (-0.6244103773584906, 472),
 (0.0005002501250625312, 329)]

In [556]:
# intersection = calc_intersection(line_equations[0][0], line_equations[0][1], line_equations[1][0], line_equations[1][1])
# cv2.circle(test_image, (int(intersection[0]), int(intersection[1])), 1, (0, 0, 255), -1)
# showImage(test_image)

perimeter_points = []
image_height, image_width, _ = test_image.shape
for ind, line_one in enumerate(line_equations):
    for line_two in line_equations[ind+1:]:
        perimeter_point = calc_intersection(line_one[0], line_one[1], line_two[0], line_two[1])
        print(perimeter_point)
        if perimeter_point[0] == 0 and perimeter_point[1] == 0:
            print("This perimiter point doesnt matter ?!")
        elif perimeter_point[0] >= 0 and perimeter_point[0] <= image_width and perimeter_point[1] >= 0 and perimeter_point[1] <= image_height:
            perimeter_points.append((round(perimeter_point[0]), round(perimeter_point[1])))

# for i in range(len(line_equations)):
#     if i+1 == len(line_equations):
#         perimeter_point = calc_intersection(line_equations[i][0], line_equations[i][1], line_equations[0][0], line_equations[0][1])
#     else:
#         perimeter_point = calc_intersection(line_equations[i][0], line_equations[i][1], line_equations[i+1][0], line_equations[i+1][1])
#     perimeter_points.append(perimeter_point)

print(f"Perimeter points: {perimeter_points}\n Length: {len(perimeter_points)}")

[511.94848566 548.32399515]
[-232.35790793  376.29632328]
[-1428.63920177    99.80603965]
[1431.01248386  760.74248471]
[ 49.09207797 441.34639707]
[-437.94122012  328.78091985]
[2720.0773389   118.40288305]
[1327.41855569  389.5525544 ]
[1018.11278721  449.77416681]
[-409.57751317  727.74444956]
[1634.22519136  329.81752136]
[550.70604534 307.89615869]
[800.7307903  286.05664783]
[215.9903705  337.13337125]
[307.3432928  329.15374852]
[874.99723542 341.98914598]
[304.30085586 281.99138776]
[755.03771738 329.37770771]
[572.7588032  114.36345956]
[858.32053447 329.42937495]
[228.83272217 329.1144736 ]
Perimeter points: [(512, 548), (49, 441), (1018, 450), (551, 308), (801, 286), (216, 337), (307, 329), (875, 342), (304, 282), (755, 329), (573, 114), (858, 329), (229, 329)]
 Length: 13


### Finding the cloesest edge points with the coordinates found in the intersections (and setting a max distance so we get rid of non-perimeter points)

In [557]:
len(perimeter_points)

13

In [558]:
# Invert the binary image (edges)
inverted_edges = 255 - edges

# Apply distance transform
dist_transform = cv2.distanceTransform(inverted_edges, cv2.DIST_L2, 5)

good_pts = []

dist_threshold = 25

# For each point, find the distance to the closest white pixel
while len(good_pts) < 6:
    good_pts = []
    for i in range(len(perimeter_points)):
        x, y = perimeter_points[i]
        dist_to_edge = dist_transform[y, x]
        if dist_to_edge < dist_threshold:
            if len(good_pts) < 6:
                good_pts.append(perimeter_points[i])
                # perimeter_points = perimeter_points[0:i] + perimeter_points[i+1:]
    print(len(good_pts))
    dist_threshold += 5
good_pts
colors = [(0,0,255), (51, 153, 255), (0,255,255), (0,255,0), (255,0,0), (255, 0, 127)]
for i in range(len(good_pts)):
    cv2.circle(test_image, (round(good_pts[i][0]), round(good_pts[i][1])), 5, colors[i], -1)
showImage(test_image)

6


In [559]:
str_colors = ["red", "orange", "yellow", "green", "blue", "purple"]
print(len(good_pts), len(str_colors))
for i in range(len(good_pts)):
    print(f"coord: {good_pts[i]} | color: {str_colors[i]}")

6 6
coord: (512, 548) | color: red
coord: (49, 441) | color: orange
coord: (1018, 450) | color: yellow
coord: (551, 308) | color: green
coord: (216, 337) | color: blue
coord: (307, 329) | color: purple


In [560]:
test_image.shape

(1052, 1052, 3)

### Order the points via angle from the centroid

In [561]:
centroid = np.mean(good_pts, axis=0)
sorted_points = sorted(good_pts, key=lambda point: np.arctan2(point[1] - centroid[1], point[0] - centroid[0]))
for i, point in enumerate(sorted_points):
    pass
    # cv2.circle(test_image, point, 20, colors[i], -1)

# showImage(test_image)


In [562]:
test_image.shape

(1052, 1052, 3)

# Perform Homography on the image to get a top down view

In [563]:
sorted_points

[(216, 337), (307, 329), (551, 308), (1018, 450), (512, 548), (49, 441)]

## The source points are the perimiter points that have been previously found

In [564]:
src_points = np.array(sorted_points)

## The destination points are just a correctly oriented hexagon within the 1052x1052 bounds of the image

Things that need to be done to get these points:
* calculate the center of the ideal image
* make sure the orientation is correct (might need to be rotated 90 degrees)
* init the rotation matrix
* find the points of a hexagon in a space of the image's dimensions
    * `np.array([(center[0] + R * np.cos(2 * np.pi / 6 * i), center[1] + R * np.sin(2 * np.pi / 6 * i)) for i in range(6)])`
* add these found *ideal* points to a new list in the correct format
* make sure `dst_points` and `src_points` are numpy arrays for `cv2.findHomography()`
* use `cv2.findHomography()` to calculate the rotation matrix
* use `cv2.warpHomography()` with the src and dst points to get the top-down image of a catan board

In [565]:
# Center of an ideal image
R = 526
center = np.array([R, R])

In [566]:
# Angle of rotation (this method finds points rotated 90 degrees from where we want them)
rotate = True
theta = np.pi / 2

In [567]:
# Init the rotation matrix
rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                            [np.sin(theta), np.cos(theta)]])
rotation_matrix

array([[ 6.123234e-17, -1.000000e+00],
       [ 1.000000e+00,  6.123234e-17]])

In [568]:
# Calculate the ideal points of a hexagon
hexagon_points = np.array([(center[0] + R * np.cos(2 * np.pi / 6 * i), center[1] + R * np.sin(2 * np.pi / 6 * i)) for i in range(6)])
hexagon_points

array([[1052.        ,  526.        ],
       [ 789.        ,  981.52936239],
       [ 263.        ,  981.52936239],
       [   0.        ,  526.        ],
       [ 263.        ,   70.47063761],
       [ 789.        ,   70.47063761]])

In [569]:
# Get these found points into the correct format
dst_points = []

for point in hexagon_points:
    translated_point = point - center
    rotated_point = np.dot(rotation_matrix, translated_point)
    dst_points.append([int(rotated_point[0]+R), int(rotated_point[1]+R)])

dst_points = np.array(dst_points)

print(dst_points)

[[ 526 1052]
 [  70  789]
 [  70  263]
 [ 525    0]
 [ 981  262]
 [ 981  788]]


In [570]:
src_points, dst_points

(array([[ 216,  337],
        [ 307,  329],
        [ 551,  308],
        [1018,  450],
        [ 512,  548],
        [  49,  441]]),
 array([[ 526, 1052],
        [  70,  789],
        [  70,  263],
        [ 525,    0],
        [ 981,  262],
        [ 981,  788]]))

In [571]:
# Compute homography matrix
H, _ = cv2.findHomography(src_points, dst_points)

# Apply homography to warp the real test_image to the ideal Catan board's perspective
warped_image = cv2.warpPerspective(test_image, H, (1052, 1052))

showImage(warped_image, "Homographied!")
